In [1]:
from nltk import FreqDist
import numpy as np
import re
import nltk
from konlpy.tag import Okt  # 한글 형태소 분석기 추가
from sklearn.utils.extmath import randomized_svd
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# NLTK 데이터 다운로드
try:
    nltk.download('stopwords', quiet=True)
except:
    pass

In [2]:
korean_stopwords = [
    '이', '그', '저', '것', '들', '는', '은', '이', '가', '을', '를', 
    '에', '의', '와', '과', '도', '만', '에서', '로', '으로', '부터', 
    '까지', '이다', '하다', '되다', '있다', '없다', '같다', '다르다',
    '그리고', '그런데', '하지만', '그러나', '또한', '그래서', '따라서',
    '그냥', '정말', '진짜', '매우', '너무', '아주', '좀', '조금',
    '약간', '살짝', '많이', '적게', '빨리', '천천히'
]

In [8]:
def buildDict_korean(docs):
    """
    한글 문서들로부터 사전 구축
    """
    okt = Okt()
    doc_tokens = []
    
    for doc in docs:
        # 한글 형태소 분석 및 품사 태깅
        tokens = okt.morphs(doc, stem=True)  # 어간 추출 포함
        # 불용어 제거 및 한글만 필터링
        tokens = [token for token in tokens 
                 if token not in korean_stopwords 
                 and len(token) > 1  # 한 글자 단어 제거
                 and re.match(r'^[가-힣]+$', token)]  # 한글만 허용
        doc_tokens.append(tokens)
    
    # 전체 토큰들을 평면화하여 빈도 계산
    all_tokens = []
    for tokens in doc_tokens:
        all_tokens.extend(tokens)
    
    vocab = FreqDist(all_tokens)
    vocab = vocab.most_common()
    
    word_to_id = {word[0]: id for id, word in enumerate(vocab)}
    id_to_word = {id: word[0] for id, word in enumerate(vocab)}
    
    # 전체 코퍼스를 단어 ID 배열로 변환
    corpus = []
    for tokens in doc_tokens:
        for token in tokens:
            if token in word_to_id:
                corpus.append(word_to_id[token])
    
    corpus = np.array(corpus)
    return doc_tokens, corpus, word_to_id, id_to_word

In [9]:
def create_co_matrix(corpus, vocab_size, window_size=1):
    """
    동시발생 행렬 생성
    :param corpus: 말뭉치(단어 ID 목록)
    :param vocab_size: 어휘 수
    :param window_size: 윈도우 크기
    :return: 동시발생 행렬
    """
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)

    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i

            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1

            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1

    return co_matrix

In [10]:
def ppmi(C, verbose=False, eps=1e-8):
    """
    PPMI (점별 상호정보량) 생성
    :param C: 동시발생 행렬
    :param verbose: 진행 상황을 출력할지 여부
    :return: PPMI 행렬
    """
    M = np.zeros_like(C, dtype=np.float32)
    N = np.sum(C)
    S = np.sum(C, axis=0)
    total = C.shape[0] * C.shape[1]
    cnt = 0

    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            pmi = np.log2(C[i, j] * N / (S[j] * S[i]) + eps)
            M[i, j] = max(0, pmi)

            if verbose:
                cnt += 1
                if cnt % (total // 100 + 1) == 0:
                    print('%.1f%% 완료' % (100 * cnt / total))

    return M


In [11]:
def most_similar_korean(query, word_to_id, id_to_word, word_matrix, top=5):
    """
    한글 단어에 대한 유사 단어 찾기
    """
    if query not in word_to_id:
        print(f'"{query}"를 찾을 수 없습니다.')
        print(f'사전에 있는 단어들 (처음 20개): {list(word_to_id.keys())[:20]}')
        return []

    word_vector = np.array([word_matrix[word_to_id[query]]])
    word_vector = word_vector.reshape(1, -1)
    sim = cosine_similarity(word_vector, word_matrix)
    sim = sim[0]

    sim = [(id, cos) for id, cos in enumerate(sim)]
    sim = sorted(sim, key=lambda x: x[1], reverse=True)

    return sim[1:top+1]

In [12]:
if __name__ == "__main__":
    # 한글 BTS 데이터 로딩
    try:
        with open('./bts_korean.txt', 'r', encoding='utf-8') as f:
            kor_docs = f.readlines()
        
        print(f"로딩된 한글 문서 수: {len(kor_docs)}")
        
        # 처음 몇 개 문서 미리보기
        for id, doc in enumerate(kor_docs[:5]):
            print(f'[{id}] : {doc.strip()[:50]}...')
        
        print("\n=== 한글 형태소 분석 및 사전 구축 ===")
        doc_tokens, corpus, word_to_id, id_to_word = buildDict_korean(kor_docs)
        
        vocab_size = len(word_to_id)
        print(f"어휘 크기: {vocab_size}")
        print(f"코퍼스 크기: {len(corpus)}")
        
        # 상위 빈도 단어들 출력
        print("상위 빈도 단어들:")
        for i, (word, word_id) in enumerate(list(word_to_id.items())[:10]):
            print(f"{word}: {word_id}")
        
        print("\n=== 동시발생 행렬 및 PPMI 계산 ===")
        window_size = 2
        C = create_co_matrix(corpus, vocab_size, window_size)
        W = ppmi(C, verbose=True)
        
        print(f"동시발생 행렬 형태: {C.shape}")
        print(f"PPMI 행렬 형태: {W.shape}")
        
        print("\n=== SVD 차원 축소 ===")
        wordvec_size = min(100, vocab_size-1)  # 어휘 크기보다 작게 설정
        U, S, V = randomized_svd(W, n_components=wordvec_size, n_iter=5, random_state=42)
        
        print(f"단어 벡터 차원: {wordvec_size}")
        print(f"U 행렬 형태: {U.shape}")
        
        print("\n=== '월드' 단어 유사도 분석 ===")
        # '월드' 단어 검색
        similar_words = most_similar_korean('월드', word_to_id, id_to_word, U, top=10)
        
        if similar_words:
            print("'월드'와 유사한 단어들:")
            for rank, (word_id, similarity) in enumerate(similar_words, 1):
                print(f"{rank}. {id_to_word[word_id]}: {similarity:.4f}")
        
        # 다른 키워드들도 테스트
        test_words = ['음악', '사랑', '세상', '꿈', '마음']
        for test_word in test_words:
            if test_word in word_to_id:
                print(f"\n=== '{test_word}' 단어 유사도 분석 ===")
                similar = most_similar_korean(test_word, word_to_id, id_to_word, U, top=5)
                if similar:
                    for rank, (word_id, similarity) in enumerate(similar, 1):
                        print(f"{rank}. {id_to_word[word_id]}: {similarity:.4f}")
    
    except FileNotFoundError:
        print("'bts_korean.txt' 파일을 찾을 수 없습니다.")
    
    except Exception as e:
        print(f"오류 발생: {e}")

로딩된 한글 문서 수: 81
[0] : ...
[1] : 방탄복이 총알을 막아내는 것처럼, 살아가는 동안 힘든 일을 겪는 10대, 20대가 겪는 힘...
[2] : ...
[3] : ...
[4] : 2013년 방탄소년단은 《2 COOL 4 SKOOL》을 발매하며 데뷔하였고, 그 해 신인상...

=== 한글 형태소 분석 및 사전 구축 ===
어휘 크기: 1072
코퍼스 크기: 3760
상위 빈도 단어들:
방탄소년단: 0
차트: 1
앨범: 2
빌보드: 3
발매: 4
기록: 5
미국: 6
번째: 7
뮤직: 8
싱글: 9

=== 동시발생 행렬 및 PPMI 계산 ===
1.0% 완료
2.0% 완료
3.0% 완료
4.0% 완료
5.0% 완료
6.0% 완료
7.0% 완료
8.0% 완료
9.0% 완료
10.0% 완료
11.0% 완료
12.0% 완료
13.0% 완료
14.0% 완료
15.0% 완료
16.0% 완료
17.0% 완료
18.0% 완료
19.0% 완료
20.0% 완료
21.0% 완료
22.0% 완료
23.0% 완료
24.0% 완료
25.0% 완료
26.0% 완료
27.0% 완료
28.0% 완료
29.0% 완료
30.0% 완료
31.0% 완료
32.0% 완료
33.0% 완료
34.0% 완료
35.0% 완료
36.0% 완료
37.0% 완료
38.0% 완료
39.0% 완료
40.0% 완료
41.0% 완료
42.0% 완료
43.0% 완료
44.0% 완료
45.0% 완료
46.0% 완료
47.0% 완료
48.0% 완료
49.0% 완료
50.0% 완료
51.0% 완료
52.0% 완료
53.0% 완료
54.0% 완료
55.0% 완료
56.0% 완료
57.0% 완료
58.0% 완료
59.0% 완료
60.0% 완료
61.0% 완료
62.0% 완료
63.0% 완료
64.0% 완료
65.0% 완료
66.0% 완료
67.0% 완료
68.0% 완료
69.0% 완료
70.0% 완료
71.0% 완료
72.0% 완료
73.0% 완료
74.0% 완료
75.0% 완료
76.0% 완료
77.0%